In [51]:
# prompt: extract questions from pdf using llm

# prompt: update code to use azureAi embeddings instead of openai embeddings

# Import the Azure AI library


# Create an OpenAI embedding object using your Azure API key and endpoint


# Replace the OpenAI embedding with the Azure AI embedding in your code


# The rest of your code remains the same


!pip install langchain
!pip install --upgrade pymupdf
!pip install openai
from langchain.document_loaders import PyMuPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain import OpenAI, PromptTemplate
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder



In [224]:
#Load the PDF document
loader = PyMuPDFLoader("Survey-Questionnire-Part5.pdf")
pages = loader.load()

In [225]:
pages = loader.load_and_split()
len(pages)

48

In [226]:
pages

[Document(page_content='Page 73 \n \nC4 Targets and performance \n \n \nEmissions targets \n \n \n(C4.1) Did you have an emissions target that was active in the reporting year? \n \n \nConnection to other frameworks \nSDG \nGoal 7: Affordable and clean energy   \nGoal 12: Responsible consumption and production \nTCFD \nMetrics & Targets recommended disclosure c) Describe the targets used by the organization to manage climate related risks and opportunities and \nperformance against targets.   \nS&P Global Corporate Sustainability Assessment \nTCFD Disclosure   \nNZAM (FS Only) \nCommitment  1 \nResponse options \nSelect all that apply:   \n● \nAbsolute target \n● \nIntensity target \n● \nPortfolio target [FS only]', metadata={'source': 'Survey-Questionnire-Part5.pdf', 'file_path': 'Survey-Questionnire-Part5.pdf', 'page': 0, 'total_pages': 48, 'format': 'PDF 1.7', 'title': 'Microsoft Word - CC_Questionnaire_FS_Full_Banksprogram_InvestmentCommunity_NetZeroAssetManagersinitiative_RE100_Su

In [227]:
# Split the document into smaller chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [228]:
texts

[Document(page_content='Page 73 \n \nC4 Targets and performance \n \n \nEmissions targets \n \n \n(C4.1) Did you have an emissions target that was active in the reporting year? \n \n \nConnection to other frameworks \nSDG \nGoal 7: Affordable and clean energy   \nGoal 12: Responsible consumption and production \nTCFD \nMetrics & Targets recommended disclosure c) Describe the targets used by the organization to manage climate related risks and opportunities and \nperformance against targets.   \nS&P Global Corporate Sustainability Assessment \nTCFD Disclosure   \nNZAM (FS Only) \nCommitment  1 \nResponse options \nSelect all that apply:   \n● \nAbsolute target \n● \nIntensity target \n● \nPortfolio target [FS only]', metadata={'source': 'Survey-Questionnire-Part5.pdf', 'file_path': 'Survey-Questionnire-Part5.pdf', 'page': 0, 'total_pages': 48, 'format': 'PDF 1.7', 'title': 'Microsoft Word - CC_Questionnaire_FS_Full_Banksprogram_InvestmentCommunity_NetZeroAssetManagersinitiative_RE100_Su

In [229]:
pages

[Document(page_content='Page 73 \n \nC4 Targets and performance \n \n \nEmissions targets \n \n \n(C4.1) Did you have an emissions target that was active in the reporting year? \n \n \nConnection to other frameworks \nSDG \nGoal 7: Affordable and clean energy   \nGoal 12: Responsible consumption and production \nTCFD \nMetrics & Targets recommended disclosure c) Describe the targets used by the organization to manage climate related risks and opportunities and \nperformance against targets.   \nS&P Global Corporate Sustainability Assessment \nTCFD Disclosure   \nNZAM (FS Only) \nCommitment  1 \nResponse options \nSelect all that apply:   \n● \nAbsolute target \n● \nIntensity target \n● \nPortfolio target [FS only]', metadata={'source': 'Survey-Questionnire-Part5.pdf', 'file_path': 'Survey-Questionnire-Part5.pdf', 'page': 0, 'total_pages': 48, 'format': 'PDF 1.7', 'title': 'Microsoft Word - CC_Questionnaire_FS_Full_Banksprogram_InvestmentCommunity_NetZeroAssetManagersinitiative_RE100_Su

In [230]:
# Define the OpenAI embedding model
openai_embeddings = OpenAIEmbeddings(api_key="e248b8e8dbf94849ac6888971cacc9ae", endpoint="https://openaimodelv3si.openai.azure.com/")
# embeddings = openai_embeddings

/usr/local/lib/python3.10/dist-packages/langchain_community/embeddings/openai.py:268: UserWarning: WARNING! endpoint is not default parameter.
                    endpoint was transferred to model_kwargs.
                    Please confirm that endpoint is what you intended.
  warnings.warn(


In [231]:
!pip install langchain_openai

from langchain_openai import AzureChatOpenAI
import os

os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openaimodelv3si.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "e248b8e8dbf94849ac6888971cacc9ae"

In [232]:
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()

!pip install langchain_community
!pip install faiss-cpu

from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="embed1",
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
)
vector = FAISS.from_documents(pages, embeddings)

In [233]:
llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="trial1",
)

In [234]:
# # Create the prompt template for question extraction
# prompt_template = PromptTemplate(
#     input_variables=["text"],
#     template="Extract questions from the following text: {text}",
# )

In [235]:
from langchain.chains import RetrievalQA
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

In [236]:
esg_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["text"], template="Your job is to extract all questions and rhetorics which are to be answered given in the survey which is: {text}. Respond with a python list containing all extracted questions like ['ques1','ques2']..")
)

esg_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)

In [153]:
vector

In [150]:
retriever = vector.as_retriever()

In [154]:
retriever

VectorStoreRetriever(tags=['FAISS', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x780b23bd7160>)

In [155]:
len(pages)

45

In [159]:
docs = []
for page in pages:
  docs.append(page)

In [160]:
doc

[Document(page_content='1 \nDeutsche Bank AG \nSustainability Deep Dive \nMay 20, 2021 \n– consolidated Q&A of chat questions –\nTable of contents \n1.\nSustainability Journey, Strategy and Ambition Level .............................................................. 2 \n2.\nSustainable Finance ..................................................................................................................... 9 \n3.\nManaging ESG Performance ..................................................................................................... 21 \n4.\nSustainable Finance Taxonomy ................................................................................................ 25 \n5.\nClimate Risk Management ......................................................................................................... 38 \n6.\nDiversity & Inclusion ................................................................................................................... 40', metadata={'source': 'docu

In [166]:
for doc in docs:
  print(doc.page_content)

1 
Deutsche Bank AG 
Sustainability Deep Dive 
May 20, 2021 
– consolidated Q&A of chat questions –
Table of contents 
1.
Sustainability Journey, Strategy and Ambition Level .............................................................. 2 
2.
Sustainable Finance ..................................................................................................................... 9 
3.
Managing ESG Performance ..................................................................................................... 21 
4.
Sustainable Finance Taxonomy ................................................................................................ 25 
5.
Climate Risk Management ......................................................................................................... 38 
6.
Diversity & Inclusion ................................................................................................................... 40
2 
 
  
1. Sustainability Journey, Strategy and Ambition Level 
1. 


In [237]:
pages[1]

Document(page_content='Page 74 \n● \nNo target \n \n(C4.1a) Provide details of your absolute emissions target(s) and progress made against those targets. \nQuestion dependencies \nThis question only appears if you select “Absolute target” in response to C4.1.   \n \n \nConnection to other frameworks \nSDG \nGoal 7: Affordable and clean energy   \nGoal 12: Responsible consumption and production   \nGoal 13: Climate action   \nTCFD \nMetrics & Targets recommended disclosure c) Describe the targets used by the organization to manage climate related risks and opportunities and \nperformance against targets.   \nS&P Global Corporate Sustainability Assessment \nClimate-Related Targets   \nTCFD Disclosure   \nNZAM (FS only) \nCommitment 1   \nResponse options \nPlease complete the following table. The table is displayed over several rows for readability. You are able to add rows by using the “Add Row” button \nat the bottom of the table.   \n1 \n2 \n3 \n4 \n5 \n6 \n7 \nTarget reference \nnumb

In [238]:
query = "Extract all texts which seems like questions from the following text"
question_data = []

for page in pages:
  # print(page)
  with open("Output.txt", "w") as text_file:
    text_file.write(page.page_content)
  text_data = PyMuPDFLoader("Output.txt")
  new_page = text_data.load()
  response = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=FAISS.from_documents(new_page, embeddings).as_retriever(),
    return_source_documents=False
  ).invoke(query)

  print(response['result'])

  question_data.append(response['result'])
  # response['result'].find('Q:')



"Did you have an emissions target that was active in the reporting year?"
The text provided does not contain any explicit questions.
I'm sorry, but there is no specific text provided to extract questions from. If you could provide the text you would like me to extract questions from, I'd be happy to help!
I'm sorry, but there are no explicit questions in the provided text.
Does this target cover any land-related emissions?  
Please explain target coverage and identify any exclusions  
Plan for achieving target, and progress made to the end of the List the emissions reduction initiatives which contributed most  
Base year Scope 1 emissions covered by target as % of total base year emissions in Scope 1  
Base year Scope 2 emissions covered by target as % of total base year emissions in Scope 2  
Base year Scope 3, Category […] emissions covered by target as % of total base year emissions in Scope 3, Category […] (metric tons CO2e)* [One column for each Scope 3 category]  
Base year Scope

In [217]:
num = []
for data in question_data:
  num.append(data.find("Q:"))

In [218]:
len(num)

45

In [198]:
pages[1].metadata

{'source': 'document.pdf',
 'file_path': 'document.pdf',
 'page': 1,
 'total_pages': 45,
 'format': 'PDF 1.5',
 'title': 'Sustainability Deep Dive Q&A – Consolidated chat questions and anwers',
 'author': 'Deutsche Bank AG',
 'subject': '',
 'keywords': 'Public',
 'creator': 'Microsoft® Word 2013',
 'producer': 'Microsoft® Word 2013',
 'creationDate': "D:20210616124943+02'00'",
 'modDate': "D:20210616130006+02'00'",
 'trapped': ''}

In [201]:
text_data = PyMuPDFLoader("Output.txt")
new_page = text_data.load()

In [202]:
new_page

[Document(page_content='2 \n \n  \n1. Sustainability Journey, Strategy and Ambition\nLevel \n1. \nQ: Please elaborate on your targets regarding CO2\nreduction in the company car fleet. \n(Translated from original German question: Fuhrpark /\nReduzierung CO2-Ausstoß) \nA: The goal is to reduce the overall fuel consumption\nof our fleet and a 30% reduction is a valid \nproxy for the reduction of carbon emissions. To\nachieve this, we plan to adjust the car fleet \npolicy, adapting the choice of available cars based\non sustainability criteria and incentivising \nthe use of e-cars and hybrid vehicles. Plus, we are\nassessing options to support and facilitate \nenvironmentally-friendly car usage via the policy. \n \n2. \nQ: How would you plan to implement e-mobility / CO2\nreduction in your vehicle fleet. Electric \nvehicles? Charging stations for customers / employees\nat bank locations? Car sharing? \n(Translated from original German question: Wie\nmöchten Sie Elektromobilität / CO2 -Red

In [199]:
with open("Output.txt", "w") as text_file:
    text_file.write(pages[1].page_content)

In [172]:
esg_vector_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [136]:
text

Document(page_content='1 \nDeutsche Bank AG \nSustainability Deep Dive \nMay 20, 2021 \n– consolidated Q&A of chat questions –\nTable of contents \n1.\nSustainability Journey, Strategy and Ambition Level .............................................................. 2 \n2.\nSustainable Finance ..................................................................................................................... 9 \n3.\nManaging ESG Performance ..................................................................................................... 21 \n4.\nSustainable Finance Taxonomy ................................................................................................ 25 \n5.\nClimate Risk Management ......................................................................................................... 38 \n6.\nDiversity & Inclusion ................................................................................................................... 40', metadata={'source': 'docum

In [152]:
query = "Extract all texts which seems like questions from the following text"
response = esg_vector_chain.invoke(query)
response

{'query': 'Extract all texts which seems like questions from the following text',
 'result': 'Sure, here are the texts that seem like questions from the provided context:\n\n1. Q: Do you believe that there will still be a steel, ceramics, glass or paper industry in Europe in 2050?\n2. Q: The future of all speakers was painted in pink colors. Glas phere read are unrealistic fantasy’s for me. It would be more important for me to receive answers to the planned figures for the current year. Where do all these sustainability experts at Deutsche Bank suddenly come from?\n3. Q: How do you deal with small and medium enterprises where the data collection is much harder than with listed companies?\n4. Q: For clients requesting machine and equipment financing in future, how important will it be that these machines are used to produce sustainable consumer products?\n5. Q: Is Deutsche Bank going over to a lifecycle assessment of buildings when it comes to financing construction projects? Keyword: B

In [114]:
len(response['result'])

2498

In [115]:
['How do you deal with small and medium enterprises where the data collection is much harder than with listed companies?',
 'For clients requesting machine and equipment financing in future, how important will it be that these machines are used to produce sustainable consumer products?',
 'How many of your investments are impact-orientated, focused on supporting social enterprises in emerging and frontier markets? Could you give a brief example of one of Deutsche Bank’s ongoing impact investments?',
 'Some of your international customers are looking to invest in German real estate. There is however the concern that real estate investments for non-residents of Germany or the EU are at the risk of losing them at the pleasure of mercurial entry requirements. How can your bank ensure that investors shall have automatic Permanent until Reviewed rights of entry into Deutschland for themselves or family or disclosed next of kin to make decisions in relation thereto? What value of minimum investment will ensure access with or without Citizenship of the investor?',
 '1) What assurances are in place to an International Private Banking Customer from outside EU that his investment will survive him to the benefit of his successors in title without local legal impediments in the event of death of the account holder? 2) Is there a minimum investment sum that grants a tax free residential status for investments in the bank earned from outside Deutschland?',
 'Could you shed more light on the numbers and level of sustainability expertise of Deutsche Bank\'s ESG specialists/advisors for corporate and investment banking clients?',
 'Do you believe that there will still be a steel, ceramics, glass or paper industry in Europe in 2050?',
 'The future of all speakers was painted in pink colors. Glas phere read are unrealistic fantasy’s for me. It would be more important for me to receive answers to the planned figures for the current year. Where do all these sustainability experts at Deutsche Bank suddenly come from?',
 'How do you combine sustainability with the facts • that more than 680 employees in Deutsche Bank earn more than 1 Mio. EUR (2020) and • that Deutsche Bank is still providing company SUV/cars and • that Deutsche Bank is still financing the arms industry (e. g. Rheinmetal, Heckler&Koch), oil production (e. g. Exxon in Guyana-Surinam-region) and coal power (e. g. Adani Australia), and • that Deutsche Bank had to pay billions for penalties in the last few years?']

['How do you deal with small and medium enterprises where the data collection is much harder than with listed companies?',
 'For clients requesting machine and equipment financing in future, how important will it be that these machines are used to produce sustainable consumer products?',
 'How many of your investments are impact-orientated, focused on supporting social enterprises in emerging and frontier markets? Could you give a brief example of one of Deutsche Bank’s ongoing impact investments?',
 'Some of your international customers are looking to invest in German real estate. There is however the concern that real estate investments for non-residents of Germany or the EU are at the risk of losing them at the pleasure of mercurial entry requirements. How can your bank ensure that investors shall have automatic Permanent until Reviewed rights of entry into Deutschland for themselves or family or disclosed next of kin to make decisions in relation thereto? What value of minimum inve

In [116]:
len(['How do you deal with small and medium enterprises where the data collection is much harder than with listed companies?',
 'For clients requesting machine and equipment financing in future, how important will it be that these machines are used to produce sustainable consumer products?',
 'How many of your investments are impact-orientated, focused on supporting social enterprises in emerging and frontier markets? Could you give a brief example of one of Deutsche Bank’s ongoing impact investments?',
 'Some of your international customers are looking to invest in German real estate. There is however the concern that real estate investments for non-residents of Germany or the EU are at the risk of losing them at the pleasure of mercurial entry requirements. How can your bank ensure that investors shall have automatic Permanent until Reviewed rights of entry into Deutschland for themselves or family or disclosed next of kin to make decisions in relation thereto? What value of minimum investment will ensure access with or without Citizenship of the investor?',
 '1) What assurances are in place to an International Private Banking Customer from outside EU that his investment will survive him to the benefit of his successors in title without local legal impediments in the event of death of the account holder? 2) Is there a minimum investment sum that grants a tax free residential status for investments in the bank earned from outside Deutschland?',
 'Could you shed more light on the numbers and level of sustainability expertise of Deutsche Bank\'s ESG specialists/advisors for corporate and investment banking clients?',
 'Do you believe that there will still be a steel, ceramics, glass or paper industry in Europe in 2050?',
 'The future of all speakers was painted in pink colors. Glas phere read are unrealistic fantasy’s for me. It would be more important for me to receive answers to the planned figures for the current year. Where do all these sustainability experts at Deutsche Bank suddenly come from?',
 'How do you combine sustainability with the facts • that more than 680 employees in Deutsche Bank earn more than 1 Mio. EUR (2020) and • that Deutsche Bank is still providing company SUV/cars and • that Deutsche Bank is still financing the arms industry (e. g. Rheinmetal, Heckler&Koch), oil production (e. g. Exxon in Guyana-Surinam-region) and coal power (e. g. Adani Australia), and • that Deutsche Bank had to pay billions for penalties in the last few years?'])

9

In [25]:
# Create the Conversational Retrieval Chain
chain = ConversationalRetrievalChain(
    prompt=prompt_template,
    text_splitter=text_splitter,
    embeddings=embeddings,
    llm=llm,
    memory=ConversationBufferMemory(),
)

ValidationError: 7 validation errors for ConversationalRetrievalChain
combine_docs_chain
  field required (type=value_error.missing)
question_generator
  field required (type=value_error.missing)
retriever
  field required (type=value_error.missing)
embeddings
  extra fields not permitted (type=value_error.extra)
llm
  extra fields not permitted (type=value_error.extra)
prompt
  extra fields not permitted (type=value_error.extra)
text_splitter
  extra fields not permitted (type=value_error.extra)

In [20]:




# Create the Conversational Retrieval Chain
chain = ConversationalRetrievalChain(
    prompt=prompt_template,
    text_splitter=text_splitter,
    embeddings=embeddings,
    llm=llm,
    memory=ConversationBufferMemory(),
)

# Run the chain to extract questions
for text in texts:
    questions = chain.run(text)
    # Print the extracted questions
    for question in questions:
        print(question)

ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [127]:
pages

[Document(page_content='1 \nDeutsche Bank AG \nSustainability Deep Dive \nMay 20, 2021 \n– consolidated Q&A of chat questions –\nTable of contents \n1.\nSustainability Journey, Strategy and Ambition Level .............................................................. 2 \n2.\nSustainable Finance ..................................................................................................................... 9 \n3.\nManaging ESG Performance ..................................................................................................... 21 \n4.\nSustainable Finance Taxonomy ................................................................................................ 25 \n5.\nClimate Risk Management ......................................................................................................... 38 \n6.\nDiversity & Inclusion ................................................................................................................... 40 \n', metadata={'source': 'd

In [ ]:
# prompt: embeddings from azureopen ai client

from azure.ai.embeddings import OpenAIEmbeddings

# Create an OpenAI embedding object using your Azure API key and endpoint
azure_openai_embeddings = OpenAIEmbeddings(api_key="e248b8e8dbf94849ac6888971cacc9ae", endpoint="https://openaimodelv3si.openai.azure.com/")

# Use the azure_openai_embeddings object to create your OpenAI LLM model
llm = OpenAI(embeddings=azure_openai_embeddings)

# The rest of your code remains the same



In [14]:
!pip install azure
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import fitz  # PyMuPDF
import langchain

# Azure Text Analytics configuration
key = "e248b8e8dbf94849ac6888971cacc9ae"
endpoint = "https://openaimodelv3si.openai.azure.com/"
credential = AzureKeyCredential(key)
text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=credential)

# Function to extract text from a PDF document
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# Function to analyze text and extract questions using LangChain
def extract_questions(text):
    questions = []
    # Example: simple rule-based extraction of questions
    sentences = text.split('\n')  # Assuming each line is a sentence
    for sentence in sentences:
        if sentence.endswith("?"):
            questions.append(sentence)
    return questions

# Function to analyze text using Azure Text Analytics
def analyze_text_with_azure(text):
    response = text_analytics_client.analyze_sentiment(documents=[{"id": "1", "language": "en", "text": text}])
    for document in response.documents:
        print("Document Sentiment: {}".format(document.sentiment))
    return response

# Main function to process PDF and extract questions
def process_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    questions = extract_questions(text)
    analyzed_response = analyze_text_with_azure(text)
    return questions, analyzed_response

# Example usage
if __name__ == "__main__":
    pdf_path = "example.pdf"
    questions, sentiment_analysis = process_pdf(pdf_path)
    print("Extracted Questions:")
    for question in questions:
        print(question)


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


ModuleNotFoundError: No module named 'azure'